In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

sns.set_context("paper")
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches

from tqdm import tqdm # progress bars :)
tqdm.pandas()

from sourmash.lca import lca_utils
from sourmash.tax import tax_utils

In [2]:
a85_file = "big_file_cont_ani_85-90-95/gtdb-rs202.nucleotide-k31-scaled1000.recalc-ani.cut.csvani_85.txt"
a90_file = "big_file_cont_ani_85-90-95/gtdb-rs202.nucleotide-k31-scaled1000.recalc-ani.cut.csvani_90.txt"
a95_file = "big_file_cont_ani_85-90-95/gtdb-rs202.nucleotide-k31-scaled1000.recalc-ani.cut.csvani_95.txt"

#c85_file = "big_file_cont_ani_85-90-95/gtdb-rs202.nucleotide-k31-scaled1000.recalc-ani.cut.csvcont_85.txt"
#c90_file = "big_file_cont_ani_85-90-95/gtdb-rs202.nucleotide-k31-scaled1000.recalc-ani.cut.csvcont_90.txt"
#c95_file = "big_file_cont_ani_85-90-95/gtdb-rs202.nucleotide-k31-scaled1000.recalc-ani.cut.csvcont_95.txt"

# Look at 95% ANI clustering:
(k=31, scaled=100, gtdb-rs202)

### load 95% ani clustering file:

In [3]:
#a95_clusters = [x.strip().split(',') for x in open(a95_file, 'r')]
#a95_clusters[1]
a95 = pd.read_csv(a95_file, sep='\t', header=None, index_col=False, names = ["cluster_idents"])
a95_head = a95.head()
a95_head

,cluster_idents
0,"GCA_000006155,GCF_002565765,GCF_001941885,GCF_..."
1,"GCF_000178895,GCA_000007325,GCA_001296185,GCF_..."
2,"GCF_003299955,GCA_000007385,GCF_003300055,GCF_..."
3,GCA_000008085
4,"GCA_002554195,GCA_000009845,GCF_000803325,GCF_..."


### read in lineages

In [4]:
taxonomy_csv = "gtdb-rs202.taxonomy.v2.csv"
tax = pd.read_csv(taxonomy_csv)
tax['lineage'] = tax["superkingdom"] + ',' + tax["phylum"] + ',' + tax["class"] + ',' + tax["order"] + ',' + tax["family"] + ',' + tax["genus"] + ',' + tax["species"]
tax['smash_lin'] = tax['lineage'].apply(lambda x: lca_utils.make_lineage(x))
tax['split_ident'] = tax['ident'].str.split('.', expand=True)[0]
tax.head()

,ident,superkingdom,phylum,class,order,family,genus,species,lineage,smash_lin,split_ident
0,GCF_014075335.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri,"d__Bacteria,p__Proteobacteria,c__Gammaproteoba...","((superkingdom, d__Bacteria), (phylum, p__Prot...",GCF_014075335
1,GCF_002310555.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri,"d__Bacteria,p__Proteobacteria,c__Gammaproteoba...","((superkingdom, d__Bacteria), (phylum, p__Prot...",GCF_002310555
2,GCF_900013275.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri,"d__Bacteria,p__Proteobacteria,c__Gammaproteoba...","((superkingdom, d__Bacteria), (phylum, p__Prot...",GCF_900013275
3,GCF_000168095.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri,"d__Bacteria,p__Proteobacteria,c__Gammaproteoba...","((superkingdom, d__Bacteria), (phylum, p__Prot...",GCF_000168095
4,GCF_002459845.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri,"d__Bacteria,p__Proteobacteria,c__Gammaproteoba...","((superkingdom, d__Bacteria), (phylum, p__Prot...",GCF_002459845


In [5]:
taxD = tax.set_index('split_ident').to_dict()['smash_lin']

In [6]:
def count_and_find_lca_test(row, lineages=taxD):
    all_idents = row['cluster_idents']
    ident_list = all_idents.split(',')
    row['cluster_len'] = len(ident_list)
    all_lineages=[]
    for ident in ident_list:
        lineage = taxD[ident]
        all_lineages.append(lineage)
    lca_tree = lca_utils.build_tree(all_lineages)
    lca = lca_utils.find_lca(lca_tree)
    row['cluster_lca'] = lca
    row['cluster_lca_pretty'] = lca_utils.display_lineage(lca[0])
    row['lca_rank'] = lca[0][-1].rank
    print(lca[0])
    print(lca[1])
    print(row['cluster_lca_pretty'])
    return row

In [7]:
a95_head.progress_apply(count_and_find_lca_test, axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 158.15it/s]

(LineagePair(rank='superkingdom', name='d__Bacteria'), LineagePair(rank='phylum', name='p__Firmicutes'), LineagePair(rank='class', name='c__Bacilli'), LineagePair(rank='order', name='o__Bacillales'), LineagePair(rank='family', name='f__Bacillaceae_G'), LineagePair(rank='genus', name='g__Bacillus_A'))
12
d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae_G;g__Bacillus_A
(LineagePair(rank='superkingdom', name='d__Bacteria'), LineagePair(rank='phylum', name='p__Fusobacteriota'), LineagePair(rank='class', name='c__Fusobacteriia'), LineagePair(rank='order', name='o__Fusobacteriales'), LineagePair(rank='family', name='f__Fusobacteriaceae'), LineagePair(rank='genus', name='g__Fusobacterium'), LineagePair(rank='species', name='s__Fusobacterium nucleatum'))
0
d__Bacteria;p__Fusobacteriota;c__Fusobacteriia;o__Fusobacteriales;f__Fusobacteriaceae;g__Fusobacterium;s__Fusobacterium nucleatum
(LineagePair(rank='superkingdom', name='d__Bacteria'), LineagePair(rank='phylum', name='p__Pro

,cluster_idents,cluster_len,cluster_lca,cluster_lca_pretty,lca_rank
0,"GCA_000006155,GCF_002565765,GCF_001941885,GCF_...",961,"(((superkingdom, d__Bacteria), (phylum, p__Fir...",d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,genus
1,"GCF_000178895,GCA_000007325,GCA_001296185,GCF_...",12,"(((superkingdom, d__Bacteria), (phylum, p__Fus...",d__Bacteria;p__Fusobacteriota;c__Fusobacteriia...,species
2,"GCF_003299955,GCA_000007385,GCF_003300055,GCF_...",360,"(((superkingdom, d__Bacteria), (phylum, p__Pro...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,species
3,GCA_000008085,1,"(((superkingdom, d__Archaea), (phylum, p__Nano...",d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,species
4,"GCA_002554195,GCA_000009845,GCF_000803325,GCF_...",11,"(((superkingdom, d__Bacteria), (phylum, p__Fir...",d__Bacteria;p__Firmicutes;c__Bacilli;o__Achole...,genus


In [8]:
def count_and_find_lca(row, lineages=taxD):
    all_idents = row['cluster_idents']
    ident_list = all_idents.split(',')
    row['cluster_len'] = len(ident_list)
    all_lineages=[]
    for ident in ident_list:
        lineage = taxD[ident]
        all_lineages.append(lineage)
    lca_tree = lca_utils.build_tree(all_lineages)
    lca = lca_utils.find_lca(lca_tree)
    row['cluster_lca'] = lca
    row['cluster_lca_pretty'] = lca_utils.display_lineage(lca[0])
    row['lca_rank'] = lca[0][-1].rank
    return row

In [9]:
a95 = a95.progress_apply(count_and_find_lca, axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 43805/43805 [01:08<00:00, 639.99it/s]


In [10]:
a95.shape

(43805, 5)

In [11]:
a95.head()

,cluster_idents,cluster_len,cluster_lca,cluster_lca_pretty,lca_rank
0,"GCA_000006155,GCF_002565765,GCF_001941885,GCF_...",961,"(((superkingdom, d__Bacteria), (phylum, p__Fir...",d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,genus
1,"GCF_000178895,GCA_000007325,GCA_001296185,GCF_...",12,"(((superkingdom, d__Bacteria), (phylum, p__Fus...",d__Bacteria;p__Fusobacteriota;c__Fusobacteriia...,species
2,"GCF_003299955,GCA_000007385,GCF_003300055,GCF_...",360,"(((superkingdom, d__Bacteria), (phylum, p__Pro...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,species
3,GCA_000008085,1,"(((superkingdom, d__Archaea), (phylum, p__Nano...",d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,species
4,"GCA_002554195,GCA_000009845,GCF_000803325,GCF_...",11,"(((superkingdom, d__Bacteria), (phylum, p__Fir...",d__Bacteria;p__Firmicutes;c__Bacilli;o__Achole...,genus


In [12]:
# total identifiers we're looking at:
total_n_idents = a95['cluster_len'].sum()
total_n_idents

258406

In [13]:
# how many are singleton clusters? ~27k
a95[a95['cluster_len'] == 1]

,cluster_idents,cluster_len,cluster_lca,cluster_lca_pretty,lca_rank
3,GCA_000008085,1,"(((superkingdom, d__Archaea), (phylum, p__Nano...",d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,species
14,GCA_000015805,1,"(((superkingdom, d__Archaea), (phylum, p__Ther...",d__Archaea;p__Thermoproteota;c__Thermoproteia;...,species
22,GCA_000024525,1,"(((superkingdom, d__Bacteria), (phylum, p__Bac...",d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,species
27,GCA_000145985,1,"(((superkingdom, d__Archaea), (phylum, p__Ther...",d__Archaea;p__Thermoproteota;c__Thermoproteia;...,species
30,GCA_000147015,1,"(((superkingdom, d__Bacteria), (phylum, p__Pro...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,species
...,...,...,...,...,...
43799,GCF_903970965,1,"(((superkingdom, d__Bacteria), (phylum, p__Pro...",d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,species
43801,GCF_903994035,1,"(((superkingdom, d__Bacteria), (phylum, p__Fir...",d__Bacteria;p__Firmicutes;c__Bacilli;o__Staphy...,species
43802,GCF_903994045,1,"(((superkingdom, d__Bacteria), (phylum, p__Fir...",d__Bacteria;p__Firmicutes;c__Bacilli;o__Staphy...,species
43803,GCF_904061905,1,"(((superkingdom, d__Bacteria), (phylum, p__Pro...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,species


In [14]:
# how many have lca rank of species?
a95[a95['lca_rank'] == 'species']

,cluster_idents,cluster_len,cluster_lca,cluster_lca_pretty,lca_rank
1,"GCF_000178895,GCA_000007325,GCA_001296185,GCF_...",12,"(((superkingdom, d__Bacteria), (phylum, p__Fus...",d__Bacteria;p__Fusobacteriota;c__Fusobacteriia...,species
2,"GCF_003299955,GCA_000007385,GCF_003300055,GCF_...",360,"(((superkingdom, d__Bacteria), (phylum, p__Pro...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,species
3,GCA_000008085,1,"(((superkingdom, d__Archaea), (phylum, p__Nano...",d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,species
5,"GCA_013178385,GCA_000010565",2,"(((superkingdom, d__Bacteria), (phylum, p__Fir...",d__Bacteria;p__Firmicutes_B;c__Desulfotomaculi...,species
7,"GCA_000013525,GCF_014050235,GCA_000167435,GCF_...",2136,"(((superkingdom, d__Bacteria), (phylum, p__Fir...",d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,species
...,...,...,...,...,...
43800,"GCF_903986915,GCF_903986855",2,"(((superkingdom, d__Bacteria), (phylum, p__Pro...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,species
43801,GCF_903994035,1,"(((superkingdom, d__Bacteria), (phylum, p__Fir...",d__Bacteria;p__Firmicutes;c__Bacilli;o__Staphy...,species
43802,GCF_903994045,1,"(((superkingdom, d__Bacteria), (phylum, p__Fir...",d__Bacteria;p__Firmicutes;c__Bacilli;o__Staphy...,species
43803,GCF_904061905,1,"(((superkingdom, d__Bacteria), (phylum, p__Pro...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,species


In [15]:
# lca rank --  cluster length distribution
a95.groupby("lca_rank")['cluster_len'].describe()

,count,mean,std,min,25%,50%,75%,max
lca_rank,,,,,,,,
family,3.0,329.000000,297.544955,5.0,198.5,392.0,491.0,590.0
genus,2237.0,54.978543,660.263627,2.0,3.0,5.0,11.0,23509.0
order,1.0,1474.000000,NaN,1474.0,1474.0,1474.0,1474.0,1474.0
species,41563.0,3.198903,71.723025,1.0,1.0,1.0,2.0,12022.0
superkingdom,1.0,2.000000,NaN,2.0,2.0,2.0,2.0,2.0


In [16]:
a95[a95["lca_rank"]=="family"]

,cluster_idents,cluster_len,cluster_lca,cluster_lca_pretty,lca_rank
447,"GCF_007896815,GCA_900554435,GCA_001915605,GCF_...",392,"(((superkingdom, d__Bacteria), (phylum, p__Bac...",d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,family
717,"GCF_009648935,GCA_012641175,GCF_008370135,GCA_...",590,"(((superkingdom, d__Bacteria), (phylum, p__Pro...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,family
8039,"GCF_004124535,GCA_012965015,GCA_012964125,GCA_...",5,"(((superkingdom, d__Bacteria), (phylum, p__Mar...",d__Bacteria;p__Marinisomatota;c__Marinisomatia...,family


In [17]:
a95[a95["lca_rank"]=="order"]

,cluster_idents,cluster_len,cluster_lca,cluster_lca_pretty,lca_rank
245,"GCF_014057765,GCF_014057775,GCF_014057805,GCF_...",1474,"(((superkingdom, d__Bacteria), (phylum, p__Pro...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,order


### BUT, superkindom cluster is actually is species-level!!

In [18]:
a95[a95["lca_rank"]=="superkingdom"]

,cluster_idents,cluster_len,cluster_lca,cluster_lca_pretty,lca_rank
34503,"GCF_003265155,GCF_002705755",2,"(((superkingdom, d__Bacteria),), 2)",d__Bacteria,superkingdom


online:
GCF_003265155: d__Bacteria; p__Actinobacteriota; c__Actinomycetia; o__Actinomycetales; f__Microbacteriaceae; g__Microbacterium; s__Microbacterium esteraromaticum_A
GCF_002705755: d__Bacteria; p__Actinobacteriota; c__Actinomycetia; o__Actinomycetales; f__Microbacteriaceae; g__Microbacterium; s__Microbacterium esteraromaticum_A

..which is a **species** cluster!

### check gtdb-rs202 taxonomy file: 

In [19]:
tax[tax["split_ident"] == "GCF_003265155"]

,ident,superkingdom,phylum,class,order,family,genus,species,lineage,smash_lin,split_ident
242915,GCF_003265155.1,d__Bacteria,p__Firmicutes,c__Bacilli,o__Mycoplasmatales,f__Mycoplasmoidaceae,g__Eperythrozoon_A,s__Eperythrozoon_A wenyonii_A,"d__Bacteria,p__Firmicutes,c__Bacilli,o__Mycopl...","((superkingdom, d__Bacteria), (phylum, p__Firm...",GCF_003265155


In [20]:
tax[tax["split_ident"] == "GCF_002705755"]

,ident,superkingdom,phylum,class,order,family,genus,species,lineage,smash_lin,split_ident
239626,GCF_002705755.3,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Actinomycetales,f__Microbacteriaceae,g__Microbacterium,s__Microbacterium esteraromaticum_A,"d__Bacteria,p__Actinobacteriota,c__Actinomycet...","((superkingdom, d__Bacteria), (phylum, p__Acti...",GCF_002705755


In [21]:
#check an easy two-cluster:
ex_two_cluster = a95[a95["cluster_idents"] == "GCF_902158745,GCF_902158735"]
ex_two_cluster

,cluster_idents,cluster_len,cluster_lca,cluster_lca_pretty,lca_rank
43553,"GCF_902158745,GCF_902158735",2,"(((superkingdom, d__Archaea), (phylum, p__Halo...",d__Archaea;p__Halobacteriota;c__Methanoliparia...,genus


In [22]:
list(ex_two_cluster["cluster_lca"])

[((LineagePair(rank='superkingdom', name='d__Archaea'),
   LineagePair(rank='phylum', name='p__Halobacteriota'),
   LineagePair(rank='class', name='c__Methanoliparia'),
   LineagePair(rank='order', name='o__Methanoliparales'),
   LineagePair(rank='family', name='f__Methanoliparaceae'),
   LineagePair(rank='genus', name='g__Methanolliviera')),
  2)]